In [39]:
import requests
import json
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
from config import api_key

blockbusters_df = pd.read_csv("data/blockbusters.csv")
moviebudget_df= pd.read_csv("data/Movie_budget.csv")

movies_df = pd.merge(moviebudget_df,blockbusters_df, how = "right", on = "Title")
movies_df = movies_df.dropna(axis=0)
movies_df = movies_df.drop(["Worldwide_gross"], axis=1)
movies_df

,Rank,Release Date,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,length,rank_in_year,rating,Studio,Year
0,48.0,13-Feb-18,Black Panther,200000000,700059566,1336494320,Action,Adventure,Drama,7.4,135,1,PG-13,Walt Disney Pictures,2018
1,5.0,25-Apr-18,Avengers: Infinity War,300000000,678815482,2048359754,Action,Adventure,Sci-Fi,8.5,156,2,PG-13,Walt Disney Pictures,2018
2,50.0,15-Jun-18,Incredibles 2,200000000,608581744,1242805359,Animation,Action,Adventure,7.8,118,3,PG,Pixar,2018
3,144.0,6-Jun-18,Jurassic World: Fallen Kingdom,170000000,417719760,1308323302,Action,Adventure,Drama,6.2,129,4,PG-13,Universal Pictures,2018
7,11.0,23-May-18,Solo: A Star Wars Story,275000000,213767512,393151347,Action,Adventure,Drama,7.0,135,8,PG-13,Walt Disney Pictures,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,877.0,1-Jun-90,Total Recall,65000000,119394839,261399830,Action,Sci-Fi,Mystery,7.5,113,5,R,Sony Pictures,1990
293,313.0,8-Aug-14,Teenage Mutant Ninja Turtles,125000000,191204754,485004754,Comedy,Adventure,Action,6.7,93,9,PG,Warner Bros,1990
374,886.0,18-Dec-14,Annie,65000000,85911262,139829625,Family,Drama,Comedy,6.5,127,10,PG,Columbia Pictures,1982
431,44.0,14-Dec-05,King Kong,207000000,218080025,550517357,Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976


In [42]:
#Retrieving Data from OMDb 
#Adding director and actors
title= movies_df["Title"].tolist()
url = "http://www.omdbapi.com/?apikey=" + api_key + "&t="
response = []

for movie in title:
    movie_data = requests.get(url + movie).json()
    
    try:
        directors = movie_data["Director"]
    except:
        diretors = "N/A"


    response.append({ "Title":movie,
                    "Dirtector": directors})
  
    print(f"{movie} was directed by {directors}")



Black Panther was directed by Ryan Coogler
Avengers: Infinity War was directed by Anthony Russo, Joe Russo
Incredibles 2 was directed by Brad Bird
Jurassic World: Fallen Kingdom was directed by J.A. Bayona
Solo: A Star Wars Story was directed by J.A. Bayona
Venom was directed by Ruben Fleischer
Wonder Woman was directed by Ruben Fleischer
Jumanji: Welcome to the Jungle was directed by Ruben Fleischer
Spider-Man: Homecoming was directed by Ruben Fleischer
Thor: Ragnarok was directed by Ruben Fleischer
Justice League was directed by Ruben Fleischer
Rogue One: A Star Wars Story was directed by Ruben Fleischer
Captain America: Civil War was directed by Ruben Fleischer
Zootopia was directed by Ruben Fleischer
Batman v Superman: Dawn of Justice was directed by Zack Snyder
Sing was directed by Garth Jennings, Christophe Lourdelet
Jurassic World was directed by Colin Trevorrow
Avengers: Age of Ultron was directed by Colin Trevorrow
Furious 7 was directed by Colin Trevorrow
Minions was directed

In [43]:
director_df = pd.DataFrame(response)
director_df.to_csv("data/director.csv", index_label="Movie ID")
director_df

,Title,Dirtector
0,Black Panther,Ryan Coogler
1,Avengers: Infinity War,"Anthony Russo, Joe Russo"
2,Incredibles 2,Brad Bird
3,Jurassic World: Fallen Kingdom,J.A. Bayona
4,Solo: A Star Wars Story,J.A. Bayona
...,...,...
151,Total Recall,Steven Spielberg
152,Teenage Mutant Ninja Turtles,Jonathan Liebesman
153,Annie,John Huston
154,King Kong,Peter Jackson


In [45]:
allmovies_df = pd.merge(movies_df,director_df, how = "left", on = "Title")
allmovies_df

,Rank,Release Date,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,length,rank_in_year,rating,Studio,Year,Dirtector
0,48.0,13-Feb-18,Black Panther,200000000,700059566,1336494320,Action,Adventure,Drama,7.4,135,1,PG-13,Walt Disney Pictures,2018,Ryan Coogler
1,5.0,25-Apr-18,Avengers: Infinity War,300000000,678815482,2048359754,Action,Adventure,Sci-Fi,8.5,156,2,PG-13,Walt Disney Pictures,2018,"Anthony Russo, Joe Russo"
2,50.0,15-Jun-18,Incredibles 2,200000000,608581744,1242805359,Animation,Action,Adventure,7.8,118,3,PG,Pixar,2018,Brad Bird
3,144.0,6-Jun-18,Jurassic World: Fallen Kingdom,170000000,417719760,1308323302,Action,Adventure,Drama,6.2,129,4,PG-13,Universal Pictures,2018,J.A. Bayona
4,11.0,23-May-18,Solo: A Star Wars Story,275000000,213767512,393151347,Action,Adventure,Drama,7.0,135,8,PG-13,Walt Disney Pictures,2018,J.A. Bayona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,313.0,8-Aug-14,Teenage Mutant Ninja Turtles,125000000,191204754,485004754,Comedy,Adventure,Action,6.7,93,9,PG,Warner Bros,1990,Jonathan Liebesman
164,886.0,18-Dec-14,Annie,65000000,85911262,139829625,Family,Drama,Comedy,6.5,127,10,PG,Columbia Pictures,1982,John Huston
165,44.0,14-Dec-05,King Kong,207000000,218080025,550517357,Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976,Peter Jackson
166,44.0,14-Dec-05,King Kong,207000000,218080025,550517357,Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976,Peter Jackson


Changes in Cost Over the Years

* main - line

* 1990's

* 2000's

* 2010's

Production Costs per Genre
* pie chart - broken by genre
* pie chart - production companies

Production Cost vs Domestic and Worldwide Gross
* production cost vs domestic - scatter plot
* production cost vs worldwide

Production Cost vs Viewer Ratings
* scatter plot

Director
* bar - combined gross for movies

* top 100 movies directors